
---

# ✅ Validation Overview

### 🎯 Intent

* Define how Pydantic enforces correctness of data at different layers, ensuring every value is checked, normalized, and safe to use.
---

### 🧩 Core Components

1. **⚡ Automatic Validation**

   * Runs whenever a model is created.
   * Converts compatible types (e.g., `"1"` → `1`).
   * Raises `ValidationError` if input is invalid.

2. **🧱 Field-Level Validation**

   * Controlled by type hints + `Field(...)` constraints.
   * Example: enforce ranges, lengths, regex, etc.

3. **🧩 Model-Level Validation**

   * Use `@model_validator` for rules involving multiple fields.
   * Example: check if two passwords match.

4. **🔧 TypeAdapter Validation**

   * Validate plain Python types directly (without a model).
   * Works with lists, dicts, unions, etc.

5. **📋 ValidationError Structure**

   * Provides detailed error breakdown per field.
   * Includes error type, location, and human-readable message.

---


In [1]:
from pydantic import BaseModel, Field, ValidationError, model_validator, TypeAdapter
from typing import List
import re

# 1. ⚡ Automatic Validation
class User(BaseModel):
    id: int
    active: bool = True

# "1" → coerced to int 1, "true" → bool True
print(User(id="1", active="true"))

try:
    User(id="abc")  # invalid
except ValidationError as e:
    print(e.errors())

# 2. 🧱 Field-Level Validation
class Product(BaseModel):
    name: str = Field(min_length=3, max_length=50)
    price: float = Field(gt=0, description="must be positive")
    sku: str = Field(pattern=r"^[A-Z0-9]{6}$")  # regex pattern

print(Product(name="Book", price=9.99, sku="ABC123"))

# 3. 🧩 Model-Level Validation
class Signup(BaseModel):
    email: str
    password: str
    confirm_password: str

    @model_validator(mode="after")
    def check_passwords(self) -> "Signup":
        if self.password != self.confirm_password:
            raise ValueError("Passwords do not match")
        return self

try:
    Signup(email="test@example.com", password="123", confirm_password="321")
except ValidationError as e:
    print(e.errors())

# 4. 🔧 TypeAdapter Validation
adapter = TypeAdapter(List[int])

# validates + coerces list items
print(adapter.validate_python(["1", 2, 3]))  # → [1, 2, 3]

try:
    adapter.validate_python(["x", 2])
except ValidationError as e:
    print(e.errors())

# 5. 📋 ValidationError Structure
try:
    Product(name="AB", price=-5, sku="badsku")
except ValidationError as e:
    print("Error details:", e.errors())
    # Example:
    # [{'loc': ('name',), 'msg': 'String should have at least 3 characters', 'type': 'string_too_short'}, ...]


id=1 active=True
[{'type': 'int_parsing', 'loc': ('id',), 'msg': 'Input should be a valid integer, unable to parse string as an integer', 'input': 'abc', 'url': 'https://errors.pydantic.dev/2.11/v/int_parsing'}]
name='Book' price=9.99 sku='ABC123'
[{'type': 'value_error', 'loc': (), 'msg': 'Value error, Passwords do not match', 'input': {'email': 'test@example.com', 'password': '123', 'confirm_password': '321'}, 'ctx': {'error': ValueError('Passwords do not match')}, 'url': 'https://errors.pydantic.dev/2.11/v/value_error'}]
[1, 2, 3]
[{'type': 'int_parsing', 'loc': (0,), 'msg': 'Input should be a valid integer, unable to parse string as an integer', 'input': 'x', 'url': 'https://errors.pydantic.dev/2.11/v/int_parsing'}]
Error details: [{'type': 'string_too_short', 'loc': ('name',), 'msg': 'String should have at least 3 characters', 'input': 'AB', 'ctx': {'min_length': 3}, 'url': 'https://errors.pydantic.dev/2.11/v/string_too_short'}, {'type': 'greater_than', 'loc': ('price',), 'msg': '